## Importar librerías

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from genetic_algorithm import GeneticAlgorithmCV
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
import xgboost as xgb
from concurrent.futures import ThreadPoolExecutor, wait
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

## Cargar y preparar datos

In [2]:
def create_training_dataset(df, trade_type):
    df = df.drop_duplicates()
    # Filtrar las operaciones del tipo especificado y con profit != 0
    df_trade = df[(df['type'] == trade_type) & (df['profit'] != 0)].copy()
    # Separar en ganadoras y perdedoras
    df_winning = df_trade[df_trade['profit'] > 0]
    df_losing = df_trade[df_trade['profit'] < 0]
    n_winning = len(df_winning)
    n_losing = len(df_losing)
    print(f"Tipo de operación: {'Buy' if trade_type == 1 else 'Sell'}")
    print(f"Número de características: {df.iloc[:, :-2].shape[1]}")
    print(f"Total Ganadoras: {n_winning}")
    print(f"Total Perdedoras: {n_losing}")
    # Combinar las muestras seleccionadas
    df_training = pd.concat([df_winning, df_losing], ignore_index=True)
    # Añadir la columna 'Target' basada en el profit
    df_training['target'] = df_training['profit'].apply(lambda x: 1 if x > 0 else 0)
    # Seleccionar las columnas necesarias (todas menos las dos últimas para el conjunto principal,
    # y todas las columnas de los subconjuntos excepto la última)
    # Suponiendo que las dos últimas columnas en el conjunto principal son 'type' y 'profit'
    feature_columns = df.columns[:-2]
    df_training = df_training[feature_columns.tolist() + ['target']]
    # Mezclar los datos
    df_training = df_training.sample(frac=1).reset_index(drop=True)
    # Eliminar posibles missings
    if(df_training.isna().values.any()):
        df_training=df_training.dropna()
    # retunr df
    return df_training

In [3]:
# Cargar, limpiar y preparar datasets
def load_dataset(df):
    # Preparación de los datos de compra
    df_buy = create_training_dataset(df, trade_type=1)
    X_buy_train = df_buy.drop(columns='target')
    y_buy_train = df_buy['target']
    # Preparación de los datos de venta
    df_sell = create_training_dataset(df, trade_type=-1)
    X_sell_train = df_sell.drop(columns='target')
    y_sell_train = df_sell['target']
    return X_buy_train.values, y_buy_train.values, X_sell_train.values, y_sell_train.values

In [4]:
file_folder = r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/"
# Leer los archivos CSV
file_pattern = os.path.join(file_folder, 'training_dataset_*.csv')
df_file_path = glob.glob(file_pattern)
df = pd.read_csv(df_file_path[0])
X_buy_train, y_buy_train, X_sell_train, y_sell_train = load_dataset(df)

Tipo de operación: Buy
Número de características: 76
Total Ganadoras: 2624
Total Perdedoras: 2197
Tipo de operación: Sell
Número de características: 76
Total Ganadoras: 2457
Total Perdedoras: 1920


## Entrenamiento de los modelos

In [5]:
def train_model_buy(X_train, y_train, param_grid):
    try:
        # Definir el pipeline con placeholders (compras)
        estimator = Pipeline([
            ('scaler', RobustScaler()),
            ('reducer', KernelPCA(kernel='rbf')),
            ('classifier', xgb.XGBClassifier(eval_metric='auc', tree_method='gpu_hist', predictor='gpu_predictor', verbosity=0))
        ])
    # Crear una instancia de GeneticAlgorithmCV
        ga_search = GeneticAlgorithmCV(
            model_type={'buy'},
            estimator=estimator,
            cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
            param_grid=param_grid,
            n_jobs=-1,
            verbose=True
        )
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en train_model_buy: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    print("####################################################################")
    print("Mejor puntuación de validación para compras:", ga_search.best_score_)
    print("Mejores parámetros encontrados para compras:", ga_search.best_params_)
    print("####################################################################")
    # Retornar mejor estimador
    return ga_search.best_estimator_

In [6]:
def train_model_sell(X_train, y_train, param_grid):
    try:
        # Definir el pipeline con placeholders (ventas)
        estimator = Pipeline([
            ('scaler', RobustScaler()),
            ('reducer', KernelPCA(kernel='rbf')),
            ('classifier', xgb.XGBClassifier(eval_metric='auc', tree_method='gpu_hist', predictor='gpu_predictor', verbosity=0))
        ])
        # Crear una instancia de GeneticAlgorithmCV
        ga_search = GeneticAlgorithmCV(
            model_type={'sell'},
            estimator=estimator,
            cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
            param_grid=param_grid,
            n_jobs=-1,
            verbose=True
        )
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en train_model_buy: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    print("####################################################################")
    print("Mejor puntuación de validación para ventas:", ga_search.best_score_)
    print("Mejores parámetros encontrados para ventas:", ga_search.best_params_)
    print("####################################################################")
    # Retornar mejor estimador
    return ga_search.best_estimator_

In [7]:
n_features = df.iloc[:, :-2].shape[1]
# Definir espacio de hiperparámetros
param_grid = {
    'reducer__n_components': {'type': 'int', 'low': 1, 'high': n_features-1},
    'classifier__n_estimators': {'type': 'int', 'low': 50, 'high': 500},
    'classifier__max_depth': {'type': 'int', 'low': 3, 'high': 10},
    'classifier__learning_rate': {'type': 'float', 'low': 0.01, 'high': 0.3},
    'classifier__subsample': {'type': 'float', 'low': 0.6, 'high': 1.0},
    'classifier__colsample_bytree': {'type': 'float', 'low': 0.6, 'high': 1.0},
    'classifier__gamma': {'type': 'float', 'low': 0.0, 'high': 0.5},
    'classifier__min_child_weight': {'type': 'int', 'low': 1, 'high': 10},
    'classifier__reg_alpha': {'type': 'float', 'low': 0.0, 'high': 1.0},
    'classifier__reg_lambda': {'type': 'float', 'low': 0.0, 'high': 1.0},
    'classifier__scale_pos_weight': {'type': 'float', 'low': 1.0, 'high': 10.0}
}

In [ ]:
# Entrenar modelos simultáneamente
with ThreadPoolExecutor(max_workers=2) as executor:
    print("Esperando que las tareas finalicen...")
    # enviar tareas de entrenamiento
    future_buy = executor.submit(train_model_buy, X_buy_train, y_buy_train, param_grid)
    future_sell = executor.submit(train_model_sell, X_sell_train, y_sell_train, param_grid)
    # esperar a que todas las tareas terminen
    futures = [future_buy, future_sell]
    wait(futures)
    # Obtener resultados una vez que ambas tareas han terminado
    model_buy = future_buy.result()
    model_sell = future_sell.result()
    print("¡Todas las tareas han terminado!")

Esperando que las tareas finalicen...


Generaciones {'sell'}:   0%|          | 0/15 [00:00<?, ?gen/s]

Generaciones {'buy'}:   0%|          | 0/15 [00:00<?, ?gen/s]

## Exportar modelos a formato ONNX

In [9]:
def save_onnx_models(mql5_files_folder):
    try:
        update_registered_converter(
            xgb.XGBClassifier,
            "XGBClassifier",
            calculate_linear_classifier_output_shapes,
            convert_xgboost,
            options={'nocl': [True, False], 'zipmap': [True, False, 'columns']}
        )
        model_buy_onnx = convert_sklearn(
            model_buy,
            'pipeline_buy_xgboost',
            [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        model_sell_onnx = convert_sklearn(
            model_sell,
            'pipeline_sell_xgboost',
            [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
            f.write(model_buy_onnx.SerializeToString())
        with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
            f.write(model_sell_onnx.SerializeToString())
    except Exception as e:
        print(f"Error en exportar los modelos: {e}")
        raise
    print("Modelos ONNX exportados correctamente")

In [ ]:
save_onnx_models(r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files')